In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2

In [2]:
model = load_model("Face_Detection_Model_final.h5")

In [ ]:
labels_dict={0:'Face Covered',1:'Nose not covered',2:'Nose & Mouth not covered',3:'Chin Not covered'}
color_dict={0:(0,255,0),1:(0,0,255),2:(0,0,255),3:(0,0,255)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(im)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v for v in f]
        #Save just the rectangle faces in SubRecFaces
        face = im[y:y+h, x:x+w]
        
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        
        result = model.predict(face)[0]
        
        label=np.argmax(result)
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
        
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()